# Downloading Data using Google Earth Engine Python API

Note: Before running this notebook make sure you installed the neccesary packges and set up the authentication for GEE as described in the document.

Packages required:
```
pip install earthengine-api
pip install folium
pip install geehydro
```

In [1]:
# import packages
import ee
import folium
import geehydro
from IPython.display import Image

### Initialize the connection
When you use the API, the first thing you need to do is to initialize the connection to the server:

In [2]:
ee.Initialize()

###  Area of Interest
With satellite imagery you can investigate any spot on Earth. Select a region in the world you are interested on. Make sure that the projection for th e coordinates is in epsg:4326

In [6]:
geometry = ee.Geometry.Polygon([ [ [ 74.78773234410812, 21.0595214900118745 ], [ 74.351701190523794, 21.091442728540564 ], [ 74.38934656420032073, 21.084363329949862 ], [ 74.38561416973411, 21.0568038201353 ], [ 74.2951443234410812, 21.0460214900118745 ], [ 74.78773234410812, 21.0595214900118745 ]  ] ])

### The Sentinel 2 Collection
In GEE each collection has its own id. The snippet id for the LSentinel-2 MSI: MultiSpectral Instrument, Level-2A product is “COPERNICUS/S2_SR”.

You can have access to the Sentinel 2 collection with the following code:

In [ ]:
sentinel2_aoi = ee.ImageCollection("COPERNICUS/S2_SR")

### Filtering the collection
There are many filters you can apply to the collection. Let’s discuss the most common ones.

#### Filtering to the area of interest
The “.filterBounds()” method allows to select the geometry defined above.

#### Filtering by date range
The “.filterDate()” method allows to filter by date range.

#### Filtering by Max Percentage of Cloud
The “ee.Filter.lt(‘CLOUDY_PIXEL_PERCENTAGE’, use_scenes_with_max_cloud_percentage_of)” method allows to filter by cloud percentage.

#### Masking out clouds
Most satellite imagery comes with cloud mask that can be used to mask out clouds from imagery. The following function mask out clouds for Sentinel 2 images.

In [5]:
def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

#### Combining all filters and getting median image from all images in a date range

Filters can be combined. The following example gives you the median image from all images in a date range with maximum cloud coverage of 3% and clouds masked out

In [44]:
sentinel2_aoi = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(geometry)
sentinel2_median_image = sentinel2_aoi.filterDate('2019-05-15', '2019-05-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',3)).map(maskS2clouds).median()


### Meta-data
You can easily .getInfo() about the Landsat collection above or any particular information you might be interested in.

In [46]:
sentinel2_aoi.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32643',
   'crs_transform': [60, 0, 399960, 0, -60, 2400000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32643',
   'crs_transform': [10, 0, 399960, 0, -10, 2400000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32643',
   'crs_transform': [10, 0, 399960, 0, -10, 2400000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32643',
   'crs_transform': [10, 0, 399960, 0, -10, 2400000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

 It’s possible to be more selective and filter out the information you don’t want.

In [32]:
sentinel2_aoi.size().getInfo()

113

### Visualizing the Images

The Javascript-based code editor easily allows interactive visualization using ```Map.AddLayer()```, but unfortunately, this is not available for the standard Python API. A simple, static map can be made with both the ```.getThumbURL()``` module from ee and the native ```.display()``` module:

In [ ]:
# Visualizing the first image in the collection
rgbVis = {
    'min': 0.0,
    'max': 3000.0,
    'bands': ['B4', 'B3', 'B2'],
    'region': geometry}
first_img = ee.Image(sentinel2_mh_aoi.first())
Image(url= first_img.getThumbUrl(rgbVis))

In [23]:
center = geometry.centroid().coordinates()
mh_map = folium.Map(location=[center.get(1).getInfo(), center.get(0).getInfo()], zoom_start=10)
mh_map.addLayer(first_img, rgbVis)
mh_map

```geehydro``` allows an even greater degree of reproducibility between the Javascript and Python API. It allows for multiple layer to ve vizualize at the same time. The following code shows both the first image in the collection and the median image as well as the OSM layer and allows you to choose the one you want to display. 


In [40]:
# Use folium to visualize different data layers.
mapid = sentinel2_mh_median_image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000})
map = folium.Map(location=[center.get(1).getInfo(), center.get(0).getInfo()], zoom_start=10)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)
mapid = first_img.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='First Image',
  ).add_to(map)
map.add_child(folium.LayerControl())
map